In [1]:
import pandas as pd
import math

from io import StringIO
from IPython.lib.pretty import pprint

In [2]:
datastr = StringIO("""
Pokemon	Label	Level	SpdMod	IngMod	BerMod	SklMod	Ingredients	Berries
Pichu	08/08 Pichu Lvl: 3	3					1	8
Bonsly	08/09 Bonsly Lvl: 3, Skl: +	3				+	1	3
Spheal	08/09 Spheal Lvl: 5, Skl: -	5				-	1	7
Squirtle	08/09 Squirtle Lvl: 1	1					3	4
Pikachu	08/09 Pikachu Lvl: 5, Ing: -, Skl: +	5		-		+	1	15
Caterpie	08/09 Caterpie Lvl: 6, Ing: +	6		+			1	9
Totodile	08/08 Totodile Lvl: 6, Spd: +, Skl: -	6	+			-	2	9
Chikorita	08/08 Chikorita Lvl: 10, Ber: S, Skl: -	10			S	-	1	14
Raichu	08/11 Raichu Lvl: 12, Spd: +	12	+				3	19
Houndour	08/11 Houndour Lvl: 7, Spd: -	7	-				1	7
Slowpoke	08/11 Slowpoke Lvl: 6, Spd: -, Skl: +	6	-			+	1	3
Mankey	08/11 Mankey Lvl: 10	10					1	9
Ivysaur	08/11 Ivysaur Lvl: 13	13					4	6
Pikachu	08/14 Pikachu Lvl: 12, Spd: M, Ing: +	12	M	+			3	16
Jigglypuff	08/14 Jigglypuff Lvl: 10, Ing: -	10		-			1	5
Chikorita	08/14 Chikorita Lvl: 6, Ing: -	6		-			1	10
Spheal	08/14 Spheal Lvl: 10, Ing: +, Skl: S	10		+		S	1	6
Mankey	08/14 Mankey Lvl: 6, Ing: -, Skl: +	6		-		+	1	10
Ivysaur	08/14 Ivysaur Lvl: 12, Spd: M	12	M				4	6
Dugtrio	08/14 Dugtrio Lvl: 11, Spd: +	11	+				4	8
Wartortle	08/14 Wartortle Lvl: 12, Spd: M	12	M				5	6
Larvitar	08/14 Larvitar Lvl: 7	7					2	4
Charmander	08/14 Charmander Lvl: 9, Spd: +	9	+				3	6
Ivysaur	08/15 Ivysaur Lvl: 12, Spd: M	12	M				4	6
Dugtrio	08/15 Dugtrio Lvl: 12, Spd: +	12	+				4	8
Wartortle	08/15 Wartortle Lvl: 12, Spd: M	12	M				5	6
Gastly	08/15 Gastly Lvl: 7	7					2	5
Larvitar	08/15 Larvitar Lvl: 7	7					2	4
Raichu	08/14 Raichu Lvl: 16, Spd: +, Ber: S, Skl: -	16	+		S	-	3	29
Squirtle	08/14 Squirtle Lvl: 10	10					3	4
Butterfree	08/14 Butterfree Lvl: 10, Skl: S	10				S	2	15
Raichu	08/15 Raichu Lvl: 16, Spd: +, Ber: S	16	+		S		3	29
Squirtle	08/15 Squirtle Lvl: 10	10					3	4
Butterfree	08/15 Butterfree Lvl: 10	10					2	15
Slakoth	08/15 Slakoth Lvl: 6, Ing: -	6		-			1	8
Spheal	08/15 Spheal Lvl: 6, Spd: +, Ing: -	6	+	-			1	8
Toxicroak	08/17 Toxicroak Lvl: 15, Spd: +, Ing: -	15	+	-			3	7
Wartortle	08/17 Wartortle Lvl: 10, Spd: -, Ing: +	10	-	+			5	4
Graveler	08/17 Graveler Lvl: 16, Ing: M	16		M			5	4
Wigglytuff	08/17 Wigglytuff Lvl: 13, Spd: S-	13	S-				1	7
Pikachu	08/17 Pikachu Lvl: 13, Spd: +	13	+				2	15
Riolu	08/21 Riolu Lvl: 6	6					1	5
Geodude	08/21 Geodude Lvl: 4	4					2	3
Bellsprout	08/21 Bellsprout Lvl: 7	7					2	4
Eevee	08/21 Eevee Lvl: 4, Spd: +	4	+				1	6
Swablu	08/21 Swablu Lvl: 4, Skl: +	4				+	1	9
Charmeleon	08/21 Charmeleon Lvl: 12, Spd: M+	12	M+				5	8
Pikachu	08/25 Pikachu Lvl: 5, Ing: +	5		+			2	13
Bulbasaur	08/25 Bulbasaur Lvl: 8, Ing: +	8		+			3	4
Charmander	08/25 Charmander Lvl: 8, Ing: -, Skl: +	8		-		+	2	6
Eevee	08/25 Eevee Lvl: 9, Skl: +	9				+	1	5
Psyduck	08/25 Psyduck Lvl: 2, Ing: +, Skl: -	2		+		-	1	4
""")

data = pd.read_csv(datastr, sep='\t', header=0)
data = data.set_index (["Pokemon","Label"])
data = data.fillna("") 
data.sort_index(axis=0, ascending=True, inplace=True)
data.head(30)


data.to_pickle('./data/observations.pickle')

In [4]:
pokedex = pd.read_pickle('./data/pokedex.pickle')
pokedex_dict  = pokedex.to_dict(orient='index', index=True)
data_list  = data.reset_index().to_dict(orient='records', index=True)

In [5]:
for individual in data_list:   
    
    species = pokedex_dict[individual['Pokemon']]
    computeBonusAndSpeed(individual, species, individual)

In [6]:
computed = pd.DataFrame.from_records(data_list, index=['Pokemon',"Label"])
computed.sort_index(axis=0, ascending=True, inplace=True)

computed.to_pickle('./data/computed.pickle')

.head(30)

Level SpdMod IngMod  \
Pokemon    Label                                                          
Bellsprout 08/21 Bellsprout Lvl: 7                      7                 
Bonsly     08/09 Bonsly Lvl: 3, Skl: +                  3                 
Bulbasaur  08/25 Bulbasaur Lvl: 8, Ing: +               8             +   
Butterfree 08/14 Butterfree Lvl: 10, Skl: S            10                 
           08/15 Butterfree Lvl: 10                    10                 
Caterpie   08/09 Caterpie Lvl: 6, Ing: +                6             +   
Charmander 08/14 Charmander Lvl: 9, Spd: +              9      +          
           08/25 Charmander Lvl: 8, Ing: -, Skl: +      8             -   
Charmeleon 08/21 Charmeleon Lvl: 12, Spd: M+           12     M+          
Chikorita  08/08 Chikorita Lvl: 10, Ber: S, Skl: -     10                 
           08/14 Chikorita Lvl: 6, Ing: -               6             -   
Dugtrio    08/14 Dugtrio Lvl: 11, Spd: +               11      +          
           08/15 Dugtrio Lvl: 12, Spd: +               12      +          
Eevee      08/21 Eevee Lvl: 4, Spd: +                   4      +          
           08/25 Eevee Lvl: 9, Skl: +                   9                 
Gastly     08/15 Gastly Lvl: 7                          7                 
Geodude    08/21 Geodude Lvl: 4                         4                 
Graveler   08/17 Graveler Lvl: 16, Ing: M              16             M   
Houndour   08/11 Houndour Lvl: 7, Spd: -                7      -          
Ivysaur    08/11 Ivysaur Lvl: 13                       13                 
           08/14 Ivysaur Lvl: 12, Spd: M               12      M          
           08/15 Ivysaur Lvl: 12, Spd: M               12      M          
Jigglypuff 08/14 Jigglypuff Lvl: 10, Ing: -            10             -   
Larvitar   08/14 Larvitar Lvl: 7                        7                 
           08/15 Larvitar Lvl: 7                        7                 
Mankey     08/11 Mankey Lvl: 10                        10                 
           08/14 Mankey Lvl: 6, Ing: -, Skl: +          6             -   
Pichu      08/08 Pichu Lvl: 3                           3                 
Pikachu    08/09 Pikachu Lvl: 5, Ing: -, Skl: +         5             -   
           08/14 Pikachu Lvl: 12, Spd: M, Ing: +       12      M      +   

                                                   BerMod SklMod  Ingredients  \
Pokemon    Label                                                                
Bellsprout 08/21 Bellsprout Lvl: 7                                          2   
Bonsly     08/09 Bonsly Lvl: 3, Skl: +                         +            1   
Bulbasaur  08/25 Bulbasaur Lvl: 8, Ing: +                                   3   
Butterfree 08/14 Butterfree Lvl: 10, Skl: S                    S            2   
           08/15 Butterfree Lvl: 10                                         2   
Caterpie   08/09 Caterpie Lvl: 6, Ing: +                                    1   
Charmander 08/14 Charmander Lvl: 9, Spd: +                                  3   
           08/25 Charmander Lvl: 8, Ing: -, Skl: +             +            2   
Charmeleon 08/21 Charmeleon Lvl: 12, Spd: M+                                5   
Chikorita  08/08 Chikorita Lvl: 10, Ber: S, Skl: -      S      -            1   
           08/14 Chikorita Lvl: 6, Ing: -                                   1   
Dugtrio    08/14 Dugtrio Lvl: 11, Spd: +                                    4   
           08/15 Dugtrio Lvl: 12, Spd: +                                    4   
Eevee      08/21 Eevee Lvl: 4, Spd: +                                       1   
           08/25 Eevee Lvl: 9, Skl: +                          +            1   
Gastly     08/15 Gastly Lvl: 7                                              2   
Geodude    08/21 Geodude Lvl: 4                                             2   
Graveler   08/17 Graveler Lvl: 16, Ing: M                                   5   
Houndour   08/11 Houndour Lvl: 

In [9]:
computed.reset_index().sort_values("Label")

,Pokemon,Label,Level,SpdMod,IngMod,BerMod,SklMod,Ingredients,Berries,IngBonus,BerryBonus,Period,Help3h,IngFactor
9,Chikorita,"08/08 Chikorita Lvl: 10, Ber: S, Skl: -",10,,,S,-,1,14,1.0,3.0,4322,2.498843,1.00
27,Pichu,08/08 Pichu Lvl: 3,3,,,,,1,8,1.0,2.0,4282,2.522186,1.00
46,Totodile,"08/08 Totodile Lvl: 6, Spd: +, Skl: -",6,+,,,-,2,9,1.0,2.0,4009,2.693939,1.00
1,Bonsly,"08/09 Bonsly Lvl: 3, Skl: +",3,,,,+,1,3,1.0,1.0,6274,1.721390,1.00
5,Caterpie,"08/09 Caterpie Lvl: 6, Ing: +",6,,+,,,1,9,1.0,2.0,4356,2.479339,1.20
28,Pikachu,"08/09 Pikachu Lvl: 5, Ing: -, Skl: +",5,,-,,+,1,15,1.0,2.0,2678,4.032860,0.80
39,Spheal,"08/09 Spheal Lvl: 5, Skl: -",5,,,,-,1,7,1.0,2.0,5555,1.944194,1.00
42,Squirtle,08/09 Squirtle Lvl: 1,1,,,,,3,4,2.0,1.0,4500,2.400000,1.00
18,Houndour,"08/11 Houndour Lvl: 7, Spd: -",7,-,,,,1,7,1.0,2.0,5326,2.027788,1.00
19,Ivysaur,08/11 Ivysaur Lvl: 13,13,,,,,4,6,2.0,1.0,3222,3.351955,1.00
